In [17]:
import torch
%matplotlib inline
import torchvision.datasets as datasets
from torch.utils.data import DataLoader,random_split
from torchvision import transforms
import torch.optim as optim

In [58]:
from torch.utils.tensorboard import SummaryWriter

In [40]:
#Task 8.1
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())

In [14]:
#Task 8.2
selected_classes = [0,1,2,3,4]

indices = [i for i, label in enumerate(dataset.targets) if label in selected_classes]

subset_dataset = torch.utils.data.Subset(dataset, indices)

train_size = int(0.7 * len(subset_dataset))
val_size = int(0.15 * len(subset_dataset))
test_size = len(subset_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(subset_dataset, [train_size, val_size, test_size])

batch_size = 64

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [20]:
#Task 8.3
class SimpleNeuralNet(torch.nn.Module):
  def __init__(self, input_size, num_neurons):
    super(SimpleNeuralNet, self).__init__()
    self.flatten = torch.nn.Flatten()
    self.fc1 = torch.nn.Linear(input_size, num_neurons)
    self.fc2 = torch.nn.Linear(num_neurons, num_neurons)
    self.fc3 = torch.nn.Linear(num_neurons, num_neurons)
    self.fc4 = torch.nn.Linear(num_neurons, 10)

  def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [21]:
num_neurons = 102
input_size = 1 * 28 * 28
model = SimpleNeuralNet(input_size, num_neurons)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

In [22]:
epochs = 10

for epoch in range(epochs):
    model.train()
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Epoch {epoch+1}/{epochs}, Test Accuracy: {accuracy:.4f}')

print('Training complete.')

Epoch 1/10, Test Accuracy: 0.9630
Epoch 2/10, Test Accuracy: 0.9756
Epoch 3/10, Test Accuracy: 0.9769
Epoch 4/10, Test Accuracy: 0.9821
Epoch 5/10, Test Accuracy: 0.9858
Epoch 6/10, Test Accuracy: 0.9834
Epoch 7/10, Test Accuracy: 0.9867
Epoch 8/10, Test Accuracy: 0.9769
Epoch 9/10, Test Accuracy: 0.9874
Epoch 10/10, Test Accuracy: 0.9876
Training complete.


In [53]:
#Task 8.4
torch.manual_seed(42)

class ConvulationalNeuralNet(torch.nn.Module):
  def __init__(self):
        super(ConvulationalNeuralNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.flatten = torch.nn.Flatten()
        self.fc1 = torch.nn.Linear(64 * 7 * 7 * 16, 128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.fc3 = torch.nn.Linear(64, 10)

  def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        #print(x.size())
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [54]:
input_size = (1, 28, 28)
model = ConvulationalNeuralNet()

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

batch_size = 64

In [55]:
epochs = 10

for epoch in range(epochs):
    model.train()
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Epoch {epoch+1}/{epochs}, Test Accuracy: {accuracy:.4f}')

print('Training complete.')

Epoch 1/10, Test Accuracy: 0.9815
Epoch 2/10, Test Accuracy: 0.9845
Epoch 3/10, Test Accuracy: 0.9908
Epoch 4/10, Test Accuracy: 0.9904
Epoch 5/10, Test Accuracy: 0.9926
Epoch 6/10, Test Accuracy: 0.9937
Epoch 7/10, Test Accuracy: 0.9939
Epoch 8/10, Test Accuracy: 0.9930
Epoch 9/10, Test Accuracy: 0.9926
Epoch 10/10, Test Accuracy: 0.9911
Training complete.


In [43]:
#Task 8.5

torch.manual_seed(42)

class ConvulationalNeuralNet(torch.nn.Module):
  def __init__(self):
        super(ConvulationalNeuralNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool3 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = torch.nn.Flatten()
        self.fc1 = torch.nn.Linear(64 * 3 * 3, 128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.fc3 = torch.nn.Linear(64, 10)

  def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = self.pool3(torch.relu(self.conv3(x)))
        #print(x.size())
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

input_size = (1, 28, 28)
model = ConvulationalNeuralNet()

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

batch_size = 64

epochs = 10

for epoch in range(epochs):
    model.train()
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Epoch {epoch+1}/{epochs}, Test Accuracy: {accuracy:.4f}')

print('Training complete.')

Epoch 1/10, Test Accuracy: 0.9717
Epoch 2/10, Test Accuracy: 0.9850
Epoch 3/10, Test Accuracy: 0.9863
Epoch 4/10, Test Accuracy: 0.9919
Epoch 5/10, Test Accuracy: 0.9935
Epoch 6/10, Test Accuracy: 0.9889
Epoch 7/10, Test Accuracy: 0.9937
Epoch 8/10, Test Accuracy: 0.9954
Epoch 9/10, Test Accuracy: 0.9956
Epoch 10/10, Test Accuracy: 0.9959
Training complete.


In [56]:
#Task 8.6
torch.manual_seed(42)

class ConvulationalNeuralNet(torch.nn.Module):
    def __init__(self):
        super(ConvulationalNeuralNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = torch.nn.Dropout(0.25)
        self.conv2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = torch.nn.Dropout(0.25)
        self.conv3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool3 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout3 = torch.nn.Dropout(0.25)
        self.flatten = torch.nn.Flatten()
        self.fc1 = torch.nn.Linear(64 * 3 * 3, 128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.fc3 = torch.nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.dropout1(x)
        x = self.pool2(torch.relu(self.conv2(x)))
        x = self.dropout2(x)
        x = self.pool3(torch.relu(self.conv3(x)))
        x = self.dropout3(x)
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

input_size = (1, 28, 28)
model = ConvulationalNeuralNet()

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

batch_size = 64

epochs = 10

for epoch in range(epochs):
    model.train()
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Epoch {epoch+1}/{epochs}, Test Accuracy: {accuracy:.4f}')

print('Training complete.')

Epoch 1/10, Test Accuracy: 0.9767
Epoch 2/10, Test Accuracy: 0.9850
Epoch 3/10, Test Accuracy: 0.9898
Epoch 4/10, Test Accuracy: 0.9900
Epoch 5/10, Test Accuracy: 0.9915
Epoch 6/10, Test Accuracy: 0.9930
Epoch 7/10, Test Accuracy: 0.9935
Epoch 8/10, Test Accuracy: 0.9939
Epoch 9/10, Test Accuracy: 0.9930
Epoch 10/10, Test Accuracy: 0.9952
Training complete.


In [67]:
#Task 8.7

model = ConvulationalNeuralNet()

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

augmentation_transform = transforms.Compose([
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    transforms.RandomRotation(degrees=30),
])

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    augmentation_transform
])
writer = SummaryWriter()
epochs = 20

for epoch in range(epochs):
    model.train()
    for batch_idx, (inputs, labels) in enumerate(train_dataloader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        writer.add_scalar('Loss/Train', loss.item(), epoch * len(train_dataloader) + batch_idx)

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    writer.add_scalar('Accuracy/Validation', accuracy, epoch)
    print(f'Epoch {epoch+1}/{epochs}, Test Accuracy: {accuracy:.4f}')

print('Training complete.')
writer.close()


Epoch 1/20, Test Accuracy: 0.9850
Epoch 2/20, Test Accuracy: 0.9887
Epoch 3/20, Test Accuracy: 0.9917
Epoch 4/20, Test Accuracy: 0.9919
Epoch 5/20, Test Accuracy: 0.9935
Epoch 6/20, Test Accuracy: 0.9948
Epoch 7/20, Test Accuracy: 0.9943
Epoch 8/20, Test Accuracy: 0.9952
Epoch 9/20, Test Accuracy: 0.9956
Epoch 10/20, Test Accuracy: 0.9943
Epoch 11/20, Test Accuracy: 0.9950
Epoch 12/20, Test Accuracy: 0.9952
Epoch 13/20, Test Accuracy: 0.9956
Epoch 14/20, Test Accuracy: 0.9952
Epoch 15/20, Test Accuracy: 0.9967
Epoch 16/20, Test Accuracy: 0.9963
Epoch 17/20, Test Accuracy: 0.9976
Epoch 18/20, Test Accuracy: 0.9974
Epoch 19/20, Test Accuracy: 0.9967
Epoch 20/20, Test Accuracy: 0.9959
Training complete.
